# Join Cross-Reference and UI Wage Data to Create a View 

In [1]:
# this code loads needed packages and connect to database.
# load sqlalchemy package
import sqlalchemy

# Define connection string (Projects is the corpmdrc DSN)
connection_string = "mssql+pyodbc://@TDI"

# Create the engine connecting to the database server
sqlalchemy.create_engine(connection_string)

# Load sql magicks 
%load_ext sql

# Connect to the database server
%sql $connection_string


## Purpose

The purpose of this code is to create a view that has at least one record per quarter for each person in our cross-reference file.  Since our synthetic UI wage file covers Q1 2017 through Q1 2021, each SSN will end up with at least 17 records because we have 17 quarters of wage data. Even people who were rarely employed or never employed should end up with at least 17 records; any quarter with no UI wage data will have 0 earnings amounts imputed. People who worked more than one job in any quarter will have more than 17 records. This will be accomplished with a 3 part join.

We will create a data view that contains all of the UI wage data PLUS ZERO wages for any quarters in which the participant didn't work at all. At start those quarters of unemployment are completely missing from UI wage data files. However, it is important to account for people who never held a UI-covered job in the data to accurately estimate employment rates and earnings amounts. To do this, we will:

•	Add participants from our cross reference file (“XRef”) who never worked at all to the wage view that we are creating. The synthetic cross reference file includes identifying information for everyone in our caseload of interest. However, The UI wage file (“UI”) includes only program clients who worked in a UI-covered job during the follow-up period. 

•	Add records for individuals on the UI wage data file who did not work every quarter. The UI wage file includes earnings for individuals only for the quarters in which they worked at a UI-covered job. At the beginning of the coding process, those who never worked have zero records, those who worked only 2 quarters for 1 employer have only 2 records, and those who worked only 1 year for 1 employer have only 4 records (1 for each quarter of employment). Those who worked for 2 employers during same quarter will have 2 records for that quarter. At the end, we will have a view with 17 quarters of data for everyone in the program.


#### Print a few records from our cross-reference file before we start.

XRef is our cross-reference file, which includes person identifier and other relevant information that is important for matching to the UI wage data file and creating comparable follow-up time frames. This synthetic file includes SSN, a program start date, and a program exit date for each person. 

ProgStart is the date the client started in the program and ProgEnd is the date the client left the program 


In [2]:
%%sql
SELECT TOP 10 *
FROM dbo.XRef
ORDER BY SSN
;


 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00
100900056,2017-01-01 00:00:00,2017-06-30 00:00:00
101800112,2017-08-01 00:00:00,2018-01-28 00:00:00
102700168,2017-06-02 00:00:00,2017-11-29 00:00:00
103600224,2017-06-03 00:00:00,2017-11-30 00:00:00
104500280,2017-06-04 00:00:00,2017-12-01 00:00:00
105400336,2017-06-05 00:00:00,2017-12-02 00:00:00
106300392,2017-06-06 00:00:00,2017-12-03 00:00:00
107200448,2017-01-06 00:00:00,2017-07-05 00:00:00
108100504,2017-08-07 00:00:00,2018-02-03 00:00:00


#### Count  the number records on cross-referenct file. This file has 1 record per client 

In [3]:
%%sql
SELECT count (distinct SSN) as NumberSSN,
        count (*) as NumberRecs
FROM dbo.XRef
/* count the number SSNs and records on the XRef before we start */
;

 * mssql+pyodbc://@TDI
Done.


NumberSSN,NumberRecs
1012,1012


#### Print a few cases from our raw UI wage file before we start. We just print 2017 quarters for illustrative purposes

In [4]:
%%sql
SELECT TOP 12 * 
FROM dbo.UI
WHERE LEFT(YR_QTR,4)=2017
ORDER BY SSN,YR_QTR
/* notice that the YR_QTR is a text column. We will also need a date version of that column */
;

 * mssql+pyodbc://@TDI
Done.


SSN,YR_QTR,WAGES,empid
100000000,2017Q1,9214,000100000
100000000,2017Q2,8561,000100000
100000000,2017Q3,12550,000100000
100900056,2017Q1,5624,056100900
100900056,2017Q2,4371,056100900
100900056,2017Q3,10992,056100900
102700168,2017Q4,1324,168102700
103600224,2017Q2,7301,224103600
103600224,2017Q3,6670,224103600
103600224,2017Q4,8050,224103600


#### Calculate a few metrics to check the nature of the data on the file before we create our View

In [5]:
%%sql
SELECT count (distinct YR_QTR) as NumQTR, -- check how many quarters of wages were sent to us on the UI wage file
       count (distinct SSN) as NumSSN, -- check how many people are on UI wage file
       count (distinct empid) as NumEmployers, -- check how many employers reported wages for our clients
       sum (case when wages > 0 then 1 end ) as NumWRKQTR, -- check how many non-zero earnings records we have 
       count (*) as NumRecs -- count how many records are on the wage file. should be same as non-zero earnings records
FROM dbo.UI
;

 * mssql+pyodbc://@TDI
Done.


NumQTR,NumSSN,NumEmployers,NumWRKQTR,NumRecs
17,912,1567,8792,8792


#### Confirming what quarters of data were sent to us on the UI wage file
The query below produces a list of the quarters that are on the synthetic data file. In this case the file covers quarter 1 of 2017 through quarter 1 of 2021.

In [6]:
%%sql
SELECT YR_QTR, COUNT(*) as NumRecs
FROM dbo.UI
GROUP BY YR_QTR
ORDER BY YR_QTR
/* These distinct quarters will next be joined to every SSN on the x-ref file
to create a record for every quarter for each SSN*/

 * mssql+pyodbc://@TDI
Done.


YR_QTR,NumRecs
2017Q1,530
2017Q2,523
2017Q3,516
2017Q4,507
2018Q1,495
2018Q2,518
2018Q3,504
2018Q4,536
2019Q1,504
2019Q2,506


### Steps of Multi-Stage Merge

#### Before doing the big multi-stage merge we will step through a few of stages separately.

First we join the x-ref file to the distinct quarters to make sure each person has at least 1 record per quater.

In [7]:
%%sql
select top 17 * from
-- ADDING all RECORDS From the Cross Reference files. 
 ( SELECT SSN, ProgStart, ProgEnd
  FROM  dbo.XRef) AS XREF
  
 CROSS JOIN -- CREATING A CARTESIAN OF EVERY SSN AND EVERY QUARTER. 
           -- EVEN SSNS NOT IN THE UI WAGE FILE WILL HAVE 1 RECORD FOR EVERY QUARTER

-- ADDING 1 RECORD FOR EVERY QUARTER IN THE FOLLOW-UP. USING DATES FROM WAGE FILE.
  (SELECT DISTINCT YR_QTR
  FROM TDI.dbo.UI) AS QUARTERS
    
ORDER BY SSN, YR_QTR;


 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd,YR_QTR
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q1
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q2
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q3
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q4
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q1
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q2
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q3
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q4
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2019Q1
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2019Q2


#### Next we join these records to the UI wage data

In [8]:
%%sql
Select top 17 * from 
-- ADDING all RECORDS From the Cross Reference files. 
 ( SELECT SSN, ProgStart, ProgEnd
  FROM  dbo.XRef) AS XREF
  
 CROSS JOIN -- CREATING A CARTESIAN OF EVERY SSN AND EVERY QUARTER. 
           -- EVEN SSNS NOT IN THE UI WAGE FILE WILL HAVE 1 RECORD FOR EVERY QUARTER

-- ADDING 1 RECORD FOR EVERY QUARTER IN THE FOLLOW-UP. USING DATES FROM WAGE FILE.
  (SELECT DISTINCT YR_QTR
  FROM TDI.dbo.UI) AS QUARTERS
  
-- NOW THAT WE HAVE ONE RECORD FOR EACH QUARTER FOR EACH SSN WE MERGE ON THE ADMIN WAGES AND EMPLOYER IDS
 LEFT OUTER JOIN
 dbo.UI as WAGES -- GOING BACK TO THE WAGES FILE HERE FOR WAGE AND EMPLOYER COLUMNS NOW
  on XREF.SSN=WAGES.SSN AND QUARTERS.YR_QTR=WAGES.YR_QTR

ORDER BY XREF.SSN, QUARTERS.YR_QTR
;


 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd,YR_QTR,SSN_1,YR_QTR_1,WAGES,empid
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q1,100000000,2017Q1,9214,000100000
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q2,100000000,2017Q2,8561,000100000
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q3,100000000,2017Q3,12550,000100000
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2017Q4,None,None,None,None
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q1,None,None,None,None
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q2,None,None,None,None
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q3,None,None,None,None
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2018Q4,None,None,None,None
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2019Q1,100000000,2019Q1,3804,000100000
100000000,2017-06-01 00:00:00,2017-11-28 00:00:00,2019Q2,100000000,2019Q2,17082,001000000


In [9]:
%%sql
DROP VIEW IF EXISTS dbo.UIFULLV 
/* the drop above is executed when the view needs to be recreated */
;

 * mssql+pyodbc://@TDI
Done.


[]

#### Multi Stage Merge and View Creation

Next we put the Multi-Stage Merge demontrated above together with some column creations and save a view

1. Pull all the SSNs in our sample from the cross-reference file.
2. Pull all the distinct quarters in our follow-up period from the UI Wage file. 
3. Join those together so that every SSN has 17 records- one record per quarter.
4. Create a date version of the quarter column which comes to us as a character column on the UI wage file.
5. Merge in wage data for those in the UI wage data file (i.e. anyone who had earnings over the time period covered).
6. Impute $0 wages for any quarter in which a client does not have a record on the wage file.

In [10]:
%%sql
-- THE WAGE FILE ALWAYS CONTAINS ONLY THOSE WHO ARE EMPLOYED. 
-- WE NEED TO CREATE A FILE THAT CONTAINS EVERYONE IN OUR CROSS REFERENCE FILE, 
-- INCLUDING INDIVIDUALS WHO ARE NOT IN THE UI WAGE DATA FILE.
-- THE CODE BELOW IMPUTES ZERO EARNINGS IF THE INDIVIDUAL IS NOT ON THE UI WAGE DATA FILE.

CREATE VIEW dbo.UIFULLV as 

SELECT  XREF.SSN, -- The XREFERNCE FILE has DISTINCT SSNs for everyone enrolled
        CAST(XREF.ProgStart AS DATE) AS ProgStart, -- converting from datetime to date data type
        CAST (XREF.ProgEnd AS DATE) AS ProgEnd,  -- converting from datetime to date data type
          QUARTERS.YR_QTR,  -- DISTINCT LIST OF ALL THE WAGE QUARTERS WE COLLECTED
          CASE RIGHT(QUARTERS.YR_QTR,1) -- SELECT JUST THE QUARTER PORTION OF CHARACTER VERSION OF DATE.
        -- BELOW, WE CONVERT THE QUARTER TO THE FIRST CALENDAR DATE OF THAT QUARTER (Quarter 1 is set to January 1, etc.)
            WHEN '1' THEN CAST((LEFT(QUARTERS.YR_QTR,4)+'-01-01') AS DATE)  -- WHEN THE 1ST QUARTER SET TO JANUARY
            WHEN '2' THEN CAST((LEFT(QUARTERS.YR_QTR,4)+'-04-01') AS DATE)  -- WHEN 2ND QUARTER SET TO APRIL
            WHEN '3' THEN CAST((LEFT(QUARTERS.YR_QTR,4)+'-07-01') AS DATE)  -- WHEN 3RD QUARTER SET TO JULY
            WHEN '4' THEN CAST((LEFT(QUARTERS.YR_QTR,4)+'-10-01') AS DATE)  -- WHEN 4TH QUARTER SET TO OCTOBER
          END AS EarnQTR, -- creating a date data type version from character column:YR_QTR
          COALESCE(WAGES.WAGES,0) AS WAGES, -- if the person does not work we want a zero in wages
          WAGES.empid  -- if the person does not work the empid COLUMN will be NULL

FROM

-- ADDING all RECORDS From the Cross Reference files. 
 ( SELECT SSN, ProgStart, ProgEnd
  FROM  dbo.XRef) AS XREF
  
 CROSS JOIN -- CREATING A CARTESIAN OF EVERY SSN AND EVERY QUARTER. 
           -- EVEN SSNS NOT IN THE UI WAGE FILE WILL HAVE 1 RECORD FOR EVERY QUARTER

-- ADDING 1 RECORD FOR EVERY QUARTER IN THE FOLLOW-UP. USING DATES FROM WAGE FILE.
  (SELECT DISTINCT YR_QTR
  FROM TDI.dbo.UI) AS QUARTERS
  
-- NOW THAT WE HAVE ONE RECORD FOR EACH QUARTER FOR EACH SSN WE MERGE ON THE ADMIN WAGES AND EMPLOYER IDS
 LEFT OUTER JOIN
 dbo.UI as WAGES -- GOING BACK TO THE WAGES FILE HERE FOR WAGE AND EMPLOYER COLUMNS NOW
  on XREF.SSN=WAGES.SSN AND QUARTERS.YR_QTR=WAGES.YR_QTR

;



 * mssql+pyodbc://@TDI
Done.


[]

#### Print a few records to check creation of our wage view.

In [11]:
%%sql

SELECT TOP 12 *
FROM dbo.UIFULLV
WHERE LEFT(YR_QTR,4) = '2017'
ORDER BY SSN, YR_QTR
/* below display the 4 records for 2017 but every client will have a record for all quarters found on the wage file*/
;

 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd,YR_QTR,EarnQTR,WAGES,empid
100000000,2017-06-01,2017-11-28,2017Q1,2017-01-01,9214,000100000
100000000,2017-06-01,2017-11-28,2017Q2,2017-04-01,8561,000100000
100000000,2017-06-01,2017-11-28,2017Q3,2017-07-01,12550,000100000
100000000,2017-06-01,2017-11-28,2017Q4,2017-10-01,0,None
100900056,2017-01-01,2017-06-30,2017Q1,2017-01-01,5624,056100900
100900056,2017-01-01,2017-06-30,2017Q2,2017-04-01,4371,056100900
100900056,2017-01-01,2017-06-30,2017Q3,2017-07-01,10992,056100900
100900056,2017-01-01,2017-06-30,2017Q4,2017-10-01,0,None
101800112,2017-08-01,2018-01-28,2017Q1,2017-01-01,0,None
101800112,2017-08-01,2018-01-28,2017Q2,2017-04-01,0,None


#### Check the results of the join:
Make sure 
1. The number of clients stayed the same (1,012) from cross-reference file.
2. The number of employers stayed the same (1,567) from the wage file.
2. The number of quarters of wages greater than zero stayed the same (8,792) from the wage file.

In [12]:
%%sql
-- CHECKING HOW MANY RECORDS, SSNS, EMPLOYERS AND QUARTERS WE HAVE FROM THIS NEW FULL DATA FILE.
SELECT COUNT(*) AS NumWageRec, -- count how may records we have have now
        COUNT(DISTINCT SSN) AS NumSAMPLE, -- count that we still have the 1,012 clients we are stated with on the XRef file
          COUNT(DISTINCT empid) AS NumEMPLOYERS, -- count how many employers are on the file
          COUNT(CASE WHEN WAGES >0 THEN 1 END) AS WagesGT0, -- count how many employment quarters we have
        COUNT(DISTINCT YR_QTR) AS NumQTR  -- count how many distinct quarters are on the file
FROM dbo.UIFULLV


;

 * mssql+pyodbc://@TDI
Done.


NumWageRec,NumSAMPLE,NumEMPLOYERS,WagesGT0,NumQTR
17764,1012,1567,8792,17


#### Print one example case of a person who was never employed
This person would be on the cross-reference file but not wage file.

In [13]:
%%sql
select TOP 1 * FROM dbo.XRef -- LOOKING FOR 1 example SSN IN XREF BUT NOT IN WAGE FILE
WHERE SSN NOT IN
(SELECT SSN FROM dbo.UI)  -- ANY SSN NOT ON UI WAGE FILE
ORDER BY SSN;

 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd
200000009,2017-08-21 00:00:00,2018-02-17 00:00:00


#### Next print how this case looks on the new VIEW create.

In [14]:
%%sql
SELECT TOP 17 *
FROM dbo.UIFULLV --PRINTING WHAT 1 SSN THAT WAS IN THE XREF but not in wages files to see how it looks on our new view file
WHERE SSN IN
(select SSN
from dbo.UIFULLV
GROUP BY SSN
HAVING SUM(WAGES)= 0) -- SUB-QUERY THAT ONLY SELECTS THOSE WITH TOTAL WAGES OF ZEROS
ORDER BY SSN, YR_QTR
;

 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd,YR_QTR,EarnQTR,WAGES,empid
200000009,2017-08-21,2018-02-17,2017Q1,2017-01-01,0,None
200000009,2017-08-21,2018-02-17,2017Q2,2017-04-01,0,None
200000009,2017-08-21,2018-02-17,2017Q3,2017-07-01,0,None
200000009,2017-08-21,2018-02-17,2017Q4,2017-10-01,0,None
200000009,2017-08-21,2018-02-17,2018Q1,2018-01-01,0,None
200000009,2017-08-21,2018-02-17,2018Q2,2018-04-01,0,None
200000009,2017-08-21,2018-02-17,2018Q3,2018-07-01,0,None
200000009,2017-08-21,2018-02-17,2018Q4,2018-10-01,0,None
200000009,2017-08-21,2018-02-17,2019Q1,2019-01-01,0,None
200000009,2017-08-21,2018-02-17,2019Q2,2019-04-01,0,None


#### Now Check that All the Cases with Zero Wage records also have Zero dollars of Wages on New View File

In [15]:
%%sql
select SUM(WAGES) as "Total Wages for Those Not On UI file"
FROM dbo.UIFULLV 
WHERE SSN in
(select SSN FROM dbo.XRef /* SELECT SSN on XREF file that are not on UI file*/
EXCEPT
select SSN FROM dbo.UI)
;

 * mssql+pyodbc://@TDI
Done.


Total Wages for Those Not On UI file
0


#### Print 2 example cases of people who worked for more than one employer in the same quarter

In [16]:
%%sql
SELECT TOP 10 * FROM dbo.UIFULLV 
WHERE LEFT(YR_QTR,4) = '2017' AND SSN IN
(select SSN
from dbo.UI
WHERE LEFT(YR_QTR,4) = '2017'
GROUP BY SSN, YR_QTR -- find cases with multiple records in the same quater in 2017
HAVING COUNT(*) >1)
ORDER BY SSN, YR_QTR
;

 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd,YR_QTR,EarnQTR,WAGES,empid
107200448,2017-01-06,2017-07-05,2017Q1,2017-01-01,11349,481072004
107200448,2017-01-06,2017-07-05,2017Q1,2017-01-01,13894,448107200
107200448,2017-01-06,2017-07-05,2017Q2,2017-04-01,0,None
107200448,2017-01-06,2017-07-05,2017Q3,2017-07-01,8574,448107200
107200448,2017-01-06,2017-07-05,2017Q4,2017-10-01,0,None
109000560,2017-06-07,2017-12-04,2017Q1,2017-01-01,0,None
109000560,2017-06-07,2017-12-04,2017Q2,2017-04-01,1121,560109000
109000560,2017-06-07,2017-12-04,2017Q3,2017-07-01,11331,560109000
109000560,2017-06-07,2017-12-04,2017Q3,2017-07-01,5070,601090005
109000560,2017-06-07,2017-12-04,2017Q4,2017-10-01,0,None


#### Those with more than 1 job in a quarter should have more than 17 records on the new view file

In [17]:
%%sql
SELECT distinct COUNT(*) as "Number of Records for those who work more than one job in a quarter"
FROM dbo.UIFULLV
WHERE SSN IN
    (SELECT distinct SSN -- identify SSN with more than 1 employer in a quarter
    FROM dbo.UI
    GROUP BY SSN, YR_QTR
    HAVING COUNT(*) > 1)
GROUP BY SSN
order by 1;

 * mssql+pyodbc://@TDI
Done.


Number of Records for those who work more than one job in a quarter
18
19
20
21
